# キーワード専用引数と位置専用引数で明確さを高める

In [ ]:
# 例外を無視する動作を制御する2つのブール値引数の位置を混同しやすい
def safe_division(number, divisor, ignore_overflow, ignore_zero_division):
  try:
    return number / divisor
  except OverflowError:
    if ignore_overflow:
      return 0
    else:
      raise
  except ZeroDivisionError:
    if ignore_zero_division:
      return float('inf')
    else:
      raise

In [2]:
result = safe_division(1.0, 10**500, True, False)
print(result)

0


In [4]:
result = safe_division(1.0, 0, False, True)
print(result)

inf


In [ ]:
# キーワード引数を使うことで、位置の混同を防ぐ
def safe_division_b(number, divisor, ignore_overflow=False, ignore_zero_division=True):
  return

問題は、キーワード引数はオプションであるため、関数呼び出し元に明確化のためのキーワード引数の使用を強制することができない

In [8]:
# safe_division_b を例にする
# 呼び出し側はキーワード引数を使っても使わなくてもよい（強制力がない）
safe_division_b(1.0, 0, ignore_overflow=True)
safe_division_b(1.0, 0, False, True)

In [9]:
# もし本当に「キーワード指定をさせたい」と思ったら、
# Python では関数定義のときに特別な記法を使う必要がある
# キーワード専用引数（*）を使う
def greet(*, name, age):
    print(f"Hello {name}, you are {age} years old.")

greet(name="Alice", age=30)  # OK
greet("Alice", 30)           # エラーになる！！

# *を入れることで、その後の引数はキーワードを必ず付けないといけなくなる

Hello Alice, you are 30 years old.


TypeError: greet() takes 0 positional arguments but 2 were given

In [10]:
def safe_division_c(number, divisor, *, ignore_overflow=False, ignore_zero_division=False):
  return

In [ ]:
# 以下のような呼び出し方が可能
safe_division_c(1.0, 0, ignore_overflow=True)
safe_division_c(1.0, 0)
safe_division_c(number=1.0, divisor=0)
safe_division_c(1.0, divisor=0)

In [16]:
# 拡張 pr 好みが変わったため、最初の引数の名前を変えると問題が起こる
def safe_division_c(numerator, denominator, *, ignore_overflow=False, ignore_zero_division=False):
  return

In [17]:
# キーワードが変わったためエラーになる
safe_division_c(number=1.0, divisor=0) # エラー
safe_division_c(1.0, divisor=0) # エラー

TypeError: safe_division_c() got an unexpected keyword argument 'number'

In [18]:
# 引数リストの / 記号で位置専用引数の終わりを示す
def safe_division_d(numerator, denominator, /, *, ignore_overflow=False, ignore_zero_division=False):
  return

In [ ]:
safe_division_d(1.0, 0) # OK
safe_division_d(numerator=1.0, denominator=0) # 位置専用引数にキーワードを利用するとエラー

In [29]:
# キーワード専用引数と位置専用引数の重要な結論は、
# 引数リストの記号 / と * の間にある引数名は位置またはキーワードで渡されること
# 例: safe_division に別のオプション引数を追加し、
# 　　呼び出し元が結果を丸めるのに何桁にするか指定できるようにする
def safe_division_e(numerator, denominator, /,
                  ndigits=10, *,
                  ignore_overflow=False,
                  ignore_zero_division=False):
  try:
    fraction = numerator / denominator
    return round(fraction, ndigits)
  except OverflowError:
    if ignore_overflow:
      return 0
    else:
      raise
  except ZeroDivisionError:
    if ignore_zero_division:
      return float('inf')
    else:
      raise

In [34]:
assert safe_division_e(22, 7) == 3.1428571429
assert safe_division_e(22, 7, 2) == 3.14
assert safe_division_e(22, 7, ndigits=1) == 3.1